In [7]:
import pandas as pd
import numpy as np
from ANN.factory_model import factory_model, create_models , models_cv
from ANN.cv import ClfiCV, RegCV, evaluate
from ANN.get import get, prepro
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from collections import Counter
from torch.utils.data import TensorDataset
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
X_trn, y_trn, X_tst, y_tst = get('data/f5.csv')
#config_file = "your_config_file.json"  # Replace with your actual config file
model = factory_model(X_trn, y_trn, "config.json")

In [3]:
# Load the pre-trained weights and update the new model
state_dict = torch.load("model_6.pth")  # Load the weights from the original model
model.model.load_state_dict(state_dict, strict=False)  # Set strict to False to ignore mismatches

# Now new_model should have the weights transferred from the pre-trained model
model.model.eval()


DynamicANN(
  (layers): ModuleList(
    (0): Linear(in_features=98, out_features=64, bias=True)
    (1): Linear(in_features=64, out_features=256, bias=True)
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): Linear(in_features=256, out_features=3, bias=True)
  )
  (dropout): Dropout(p=0.2, inplace=False)
)

In [4]:
X_tst = torch.tensor(X_tst, dtype=torch.float32).to(model.device)

with torch.no_grad():
    outputs = model.model(X_tst)
    
print(outputs)

tensor([[-3.5048, -0.0121,  0.7691],
        [ 2.3927, -2.1144, -2.7770],
        [-2.8491,  0.3399,  0.0572],
        ...,
        [ 1.2409, -0.8634, -1.5170],
        [ 1.4843, -1.0891, -1.9066],
        [-3.5251,  0.2373,  0.5331]], device='cuda:0')


In [5]:
y_test_tensor = torch.tensor(y_tst, dtype=torch.long).to(model.device)

# Compare outputs with y_test_tensor
correct_predictions = torch.sum(torch.argmax(outputs, dim=1) == y_test_tensor).item()
total_samples = len(y_test_tensor)

accuracy = correct_predictions / total_samples
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 68.16%


In [8]:
# Load your training and testing data
# Replace the following lines with your actual data loading logic

num_models = 1
batch_size = 256

b = 9
eval_results = []

test_dataset = TensorDataset(torch.tensor(X_tst, dtype=torch.float32), torch.tensor(y_tst, dtype=torch.long))
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
eval_loss, eval_metrics_df = evaluate(model, nn.CrossEntropyLoss(), test_loader, 'cuda' if torch.cuda.is_available() else 'cpu')

    # Print and save evaluation metrics
print(f"Model {b} evaluation results:")
print(eval_metrics_df)

eval_metrics_df.to_csv(f'model_{b}_evaluation_metrics.csv', index=False)
print(f"Model {b} evaluation metrics saved to 'model_{b}_evaluation_metrics.csv'")

eval_results.append(eval_loss)

print("Overall evaluation results:")
for i, result in enumerate(eval_results, start=1):
    print(f"Model {b}: Loss = {result:.4f}")

Model 9 evaluation results:
   Accuracy  Precision (Class 0)  Precision (Class 1)  Precision (Class 2)  Recall (Class 0)  Recall (Class 1)  Recall (Class 2)  F1 Score (Class 0)  F1 Score (Class 1)  F1 Score (Class 2)                             Label Confusion Matrix                                   Confusion Matrix
0  0.681646             0.828697              0.77502             0.319892          0.947016          0.615457          0.438306            0.883915            0.686083            0.369852  [[[3931, 351], [95, 1698]], [[2308, 571], [122...  [[1698, 34, 61], [278, 1967, 951], [73, 537, 4...
Model 9 evaluation metrics saved to 'model_9_evaluation_metrics.csv'
Overall evaluation results:
Model 9: Loss = 0.6887


C:\Users\user\AppData\Local\Temp\ipykernel_6148\1264411198.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_dataset = TensorDataset(torch.tensor(X_tst, dtype=torch.float32), torch.tensor(y_tst, dtype=torch.long))
